# Data Analysis and Q&A Project Using a Local LLM

## Project Overview

This project requires you to perform a comprehensive analysis of a company's stock data using only the provided data sources and a local LLM. Your analysis should answer the following six questions strictly based on the supplied data and documents—no external data is allowed. All generated answers must be firmly based on the provided data, without any fabricated content. In addition, your logic must be clear, and any attribution of events must be causally linked.

---

## Provided Data

You will be provided with the following data sets:

#### Stock Price Data (Json format)
* Timeframe: Jan 22 to Feb 5
* Fields: Open, High, Low, Close, Volume

#### Quarterly Earnings Data for the Past Year (Json format)
* Contains key financial indicators (e.g., revenue, eps) for each quarter.

#### Full Earnings Transcript Call
* The complete transcript of the earnings call, including management discussions and Q&A.

#### Balance Sheet Data for the Past Year (Json format)
* Includes assets, liabilities, and shareholders' equity information.

#### News Articles
* Full text of 10 news articles related to the company during the analysis period.

---

## Questions
Using the provided data and a local LLM, you need to answer the following six questions:

1. What is the performance of the Tesla stock during this period (Jan 22 to Feb 5)?

2. Why did the price increase on Jan 30? Please provide potential factors.

3. Compared with previous quarters, how is the performance of this quarter?

4. With unsupervised Full Self Driving scheduled to launch in limited markets like Austin by June, what regulatory challenges does Tesla foresee for a nationwide or international rollout, and how is the company strategically preparing to address these hurdles?

5. What insights can be concluded from the earnings call?

6. Which key news events influenced the stock performance, and what insights do they offer?

---

## Project Requirements
- #### Data Source Restriction:
Only use the provided data and documents. No external data or information is allowed.

- #### Answer Generation:
All generated answers must strictly be based on the provided data and documents. The LLM should not "invent" information.

- #### Clear Logic and Causal Relationships:
For each question, your answers must clearly demonstrate logical reasoning, and any attribution of cause must be explicitly linked to events in the data.

- #### Prompt Design:
You must design your own prompts for calling the local LLM to ensure that the responses are generated strictly based on the analysis results.

- #### Result Evaluation:
After generating the answers, implement an evaluation step to assess whether the responses meet the above requirements in terms of data reliance, logical clarity, and correct causation.

- #### Please put the answers to these 6 questions in a dict at the end of your submitted Python nodebook file.

For example
```code
{ "Q1 answer": "Answer1", "Q2 answer": "Answer2", "Q3 answer": "Answer3", "Q4 answer4": "Answer4", "Q5 answer": "Answer5", "Q6 answer": "Answer6"}
```

## Dependencies

* Transformers
* Torch (PyTorch)
* Accelerate

In [ ]:
%pip install transformers accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Loading and Running the Local LLM

1. **Imports Transformers utilities**  
   - `AutoModelForCausalLM`: generic class for loading any GPT‑style model  
   - `AutoTokenizer`: matching tokenizer for converting text ↔ tokens  
   - `pipeline`: high‑level helper that ties model + tokenizer into one callable  

2. **Specifies the model repository**  
   ```python
   model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

# The pipeline will automatically use the model and tokenizer you just loaded
tokenizer = AutoTokenizer.from_pretrained(model_path) # Load the tokenizer

 # Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1200, # Limit the number of tokens generated
    "return_full_text": False, # Return only the generated text
    "do_sample": True, # Use sampling to generate text
    "temperature": 0.1,# Control the randomness of the output
    "repetition_penalty": 1.1,
    "top_p": 0.9, # Control the diversity of the output
    "top_k": 50, # Control the diversity of the output
}

Device set to use cuda


### Question 1:What is the performance of the Tesla stock during this period (Jan 22 to Feb 5)?

#### Prompt construction

* Here we need to monitor the performance of the tesla stock over the specified days. To monitor the performance of the stock the LLM just need to understand the how the pricing of the stock was through the given period hence why the LLM will need to see the infomation in the `prices.json` file.
* To supplement the LLM to construct its answer we will also show the information in `balancesheet.json` so it can pickup on any trends to as why stock prices deviated and change through the mentioned days

In [16]:
system_prompt = (
    "You are a professional financial analyst with deep expertise in stocks, bonds, mutual funds, and derivatives. "
    "Your responses should be data-driven, professionally rigorous, and provide clear, step-by-step explanations. "
    "Include cautionary advice regarding potential risks, but do not offer direct investment advice."
)

# Define the user prompt template
user_prompt_template = (
    "Based on the following financial data, please analyze the company's financial health and provide insights on potential risks and opportunities for future growth.\n\n"
    "Company Name: {CompanyName}\n"
    "Time Period: {TimePeriod}\n"
    "Key Financial Data:\n"
    "- Revenue: {Revenue} million USD\n"
    "- Net Income: {NetIncome} million USD\n"
    "- Debt-to-Equity Ratio: {DebtEquityRatio}%\n"
    "- Earnings Per Share (EPS): {EPS}\n\n"
    "Answer the following questions:\n"
    "1. How strong is the company's profitability? Please explain the main factors.\n"
    "2. Is the company's debt level sustainable? Are there any financial risks?\n"
    "3. What potential risks and opportunities do you foresee based on the current data?\n\n"
    "Please detail your analysis process and provide clear conclusions and recommendations."
)

# Fill in the template with specific data
filled_user_prompt = user_prompt_template.format(
    CompanyName="ABC Corporation",
    TimePeriod="Q1 2023",
    Revenue="5000",
    NetIncome="800",
    DebtEquityRatio="45",
    EPS="1.2"
)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "system", "content": "Make you final answer show the most critical and required informations the user requires. Do not use Markdown in your answer"},
    {"role": "user", "content": filled_user_prompt},
]

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])

Okay, so I need to help ABC Corporation by analyzing their financial health. They provided some key data: revenue of 5 billion USD, net income of 800 million, a debt-to-equity ratio of 45%, and EPS of 1.2. The questions are about profitability strength, sustainability of debt, and potential risks and opportunities.

First, let me think about question 1: How strong is the company's profitability? Profitability can be measured in several ways—like gross profit margin, operating margin, net profit margin. But since we don't have those numbers, maybe I should look at EPS and how it compares to industry averages. If ABC has an EPS of 1.2, which is higher than average companies, that suggests good profitability. Also, looking at revenue and net income, they're both positive, indicating strong sales and profits.

Next, question 2: Is the company's debt level sustainable? A debt-to-equity ratio of 45% means they owe more money than they have equity. That seems high. Typically, ratios above 20%

# Question 2 
### Why did the price increase on Jan 30? Please provide potential factors.

In [ ]:
import json

system_prompt_q2 = (
    "You are an expert financial analyst assistant. "
    "Your job is to explain stock price moves using only the provided data. "
    "Do not invent or fetch any outside information—everything must be grounded in the inputs."
)

user_prompt_q2 = """
Question 2: Why did Tesla’s stock price increase on January 30?

Using only the news articles data and Tesla’s Jan 30 stock data, identify all plausible factors that contributed to the price uptick. For each factor:

  • Reference the specific news item (by title and date) or data point you’re using  
  • Explain the causal link to the price move  

Do not use any external sources.  

Here are the news articles and price info:
{input_data}

After your analysis, include a short “verification checklist” confirming:
  1. Each factor is tied to a provided item  
  2. All causal links are clearly explained  
  3. No outside data was used  
"""

with open("447_dataset/news.json", "r") as f:
    news_data = json.load(f)

news_json_str = json.dumps(news_data)

# Example of how you'd fill & call it:
filled_user = user_prompt_q2.format(input_data=news_json_str)
messages = [
    {"role": "system", "content": system_prompt_q2},
    {"role": "user",   "content": filled_user},
]
output = pipe(messages, **generation_args)
print(output[0]["generated_text"])

NameError: name 'news_json_str' is not defined